In [1]:
from sklearn.datasets import make_regression
from kiwigrad import MLP, Layer, Value
from sklearn.metrics import r2_score

In [2]:
X, y = make_regression(n_samples=200, n_features=5, n_targets=1, random_state=42)
print(X.shape)
print(y.shape)
print(X[0])
print(y[0:5])

(200, 5)
(200,)
[0.95927083 0.02451017 0.49799829 1.45114361 2.15318246]
[172.54436082 -52.9529574   -4.14935233 193.99958991 -66.96262773]


In [3]:
inputs = [[Value(i) for i in elem] for elem in X]
inputs[0]

[Value(data=0.959271, grad=0.000000),
 Value(data=0.024510, grad=0.000000),
 Value(data=0.497998, grad=0.000000),
 Value(data=1.451144, grad=0.000000),
 Value(data=2.153182, grad=0.000000)]

In [4]:
class PotNet(MLP):
    def __init__(self):
        layers = [
            Layer(nin=5, nout=16, bias=True, activation="relu"),
            Layer(nin=16, nout=16, bias=True, activation="relu"),
            Layer(nin=16, nout=1, bias=True, activation="linear")
        ]
        super().__init__(layers=layers)

model = PotNet()
y_pred = model(inputs[0])
print(y_pred)

Value(data=-6.140022, grad=0.000000)


In [5]:
epochs = 100
for epoch in range(epochs):
    y_pred = []
    for i in range(len(inputs)):
        output = model(inputs[i])
        y_pred.append(output.data)
        target = Value(y[i])
        loss = output - target
        loss_e = loss * loss
        loss_e.backward()
        for p in model.parameters():
            p.data += -0.0001 * p.grad
        model.zero_grad()
    if epoch % 10 == 0:
        print(f"e: {epoch} | l: {loss.data:.2f}")

e: 0 | l: -2.58
e: 10 | l: -3.20
e: 20 | l: -1.96
e: 30 | l: -1.83
e: 40 | l: -1.62
e: 50 | l: -1.53
e: 60 | l: -1.43
e: 70 | l: -1.36
e: 80 | l: -1.27
e: 90 | l: -1.11


In [6]:
r2 = r2_score(y, y_pred)
r2

0.999931088402729

In [7]:
idx = 3
print(y[idx], y_pred[idx])

193.99958991165147 193.48169808138405
